# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output/weather.csv')
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,provideniya,64.3833,-173.3000,45.09,79,100,6.78,RU,1658936846
1,1,ushuaia,-54.8000,-68.3000,38.86,70,20,9.22,AR,1658936709
2,2,chapais,49.7834,-74.8492,69.58,66,100,3.06,CA,1658936846
3,3,sitka,57.0531,-135.3300,55.29,82,20,4.61,US,1658936847
4,4,chuy,-33.6971,-53.4616,62.28,86,77,8.14,UY,1658936847


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# heatmap of the humidity
locations = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"]
fig = gmaps.figure()


# add the heatmaplayer to the figure

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_df.loc[
    (weather_df["Max Temp"] > 10) & (weather_df["Max Temp"] < 80) &\
    (weather_df["Wind Speed"] < 10) &  (weather_df["Cloudiness"] == 0)
].dropna()

ideal_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,lure,47.7500,6.5000,79.97,34,0,6.91,FR,1658936854
31,31,bredasdorp,-34.5322,20.0403,53.28,77,0,7.85,ZA,1658936855
107,107,smithers,54.7804,-127.1743,59.32,88,0,3.44,CA,1658936632
147,147,dondo,-19.6094,34.7431,69.31,71,0,5.97,MZ,1658936893
169,169,yulara,-25.2406,130.9889,45.32,31,0,9.86,AU,1658936900


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = ideal_weather_df[['City','Country','Lat','Lng']]
hotel_df.head()

,City,Country,Lat,Lng
29,lure,FR,47.7500,6.5000
31,bredasdorp,ZA,-34.5322,20.0403
107,smithers,CA,54.7804,-127.1743
147,dondo,MZ,-19.6094,34.7431
169,yulara,AU,-25.2406,130.9889


In [17]:
hotel_df["Hotel Name"] = ''
hotel_df

/var/folders/8z/2pz_ss256bx1frzqs_0bv35h0000gn/T/ipykernel_45655/1937139935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Country,Lat,Lng,Hotel Name
29,lure,FR,47.7500,6.5000,
31,bredasdorp,ZA,-34.5322,20.0403,
107,smithers,CA,54.7804,-127.1743,
147,dondo,MZ,-19.6094,34.7431,
169,yulara,AU,-25.2406,130.9889,
174,kodiak,US,57.7900,-152.4072,
200,taos,US,36.4072,-105.5731,
266,alice springs,AU,-23.7000,133.8833,
274,mount isa,AU,-20.7333,139.5000,
297,coquimbo,CL,-29.9533,-71.3436,


In [20]:
# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #get coordinates from row
    lat = row["Lat"]
    lng = row["Lng"] 
    
    params["location"]= f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    # print['results'][0]['name']
    
    try:
      hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        
hotel_df        

Error finding lodging in maamba. Skipping...
Error finding lodging in mopipi. Skipping...


,City,Country,Lat,Lng,Hotel Name
29,lure,FR,47.7500,6.5000,Gîte Les Tourbières Belmont
31,bredasdorp,ZA,-34.5322,20.0403,Bredasdorp Country Manor
107,smithers,CA,54.7804,-127.1743,"Prestige Hudson Bay Lodge, Premier Collection"
147,dondo,MZ,-19.6094,34.7431,Pensao Casual
169,yulara,AU,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
174,kodiak,US,57.7900,-152.4072,Best Western Kodiak Inn
200,taos,US,36.4072,-105.5731,El Monte Sagrado
266,alice springs,AU,-23.7000,133.8833,Desert Palms Alice Springs
274,mount isa,AU,-20.7333,139.5000,ibis Styles Mt Isa Verona
297,coquimbo,CL,-29.9533,-71.3436,Hotel San Juan


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
marker_layer= gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))